In [ ]:
## Here I am dividing the data first based onto protected attribute value and then train two separate models

import pandas as pd
import random,time
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics

import sys
sys.path.append(os.path.abspath('..'))

from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy

In [ ]:
## Load dataset
from sklearn import preprocessing
dataset_orig = pd.read_csv('../dataset/processed.cleveland.data.csv')

## Drop NULL values
dataset_orig = dataset_orig.dropna()


## calculate mean of age column
mean = dataset_orig.loc[:,"age"].mean()
dataset_orig['age'] = np.where(dataset_orig['age'] >= mean, 1, 0)

## Make goal column binary
dataset_orig['Probability'] = np.where(dataset_orig['Probability'] > 0, 1, 0)


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)

# divide the data based on sex
dataset_orig_male , dataset_orig_female = [x for _, x in dataset_orig.groupby(dataset_orig['age'] == 0)]

print(dataset_orig_male.shape)
print(dataset_orig_female.shape)

# dataset_orig

# Train the model for > 25 age

In [ ]:
dataset_orig_male['age'] = 0
X_train_male, y_train_male = dataset_orig_male.loc[:, dataset_orig_male.columns != 'Probability'], dataset_orig_male['Probability']
# --- LSR
clf_male = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf_male.fit(X_train_male, y_train_male)

# print(X_train_male['sex'])
import matplotlib.pyplot as plt
y = np.arange(len(dataset_orig_male.columns)-1)
plt.barh(y,clf_male.coef_[0])
plt.yticks(y,dataset_orig_male.columns)
plt.show()

print(clf_male.coef_[0])

# Train the model for <25 age

In [ ]:
X_train_female, y_train_female = dataset_orig_female.loc[:, dataset_orig_female.columns != 'Probability'], dataset_orig_female['Probability']
# --- LSR
clf_female = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf_female.fit(X_train_female, y_train_female)


import matplotlib.pyplot as plt
y = np.arange(len(dataset_orig_female.columns)-1)
plt.barh(y,clf_female.coef_[0])
plt.yticks(y,dataset_orig_female.columns)
plt.show()

# Remove biased rows

In [ ]:
print(dataset_orig.shape)

df_removed = pd.DataFrame(columns=dataset_orig.columns)

for index,row in dataset_orig.iterrows():
    row_ = [row.values[0:len(row.values)-1]]
    y_male = clf_male.predict(row_)
    y_female = clf_female.predict(row_)
    if y_male[0] != y_female[0]:        
        df_removed = df_removed.append(row, ignore_index=True)
        dataset_orig = dataset_orig.drop(index)
        
print(dataset_orig.shape)
print(df_removed.shape)
print(dataset_orig.columns)

# Check Class distribution

In [ ]:
dataset_orig_male , dataset_orig_female = [x for _, x in dataset_orig.groupby(dataset_orig['age'] == 0)]

print(dataset_orig_male['Probability'].value_counts())
print(dataset_orig_female['Probability'].value_counts())

# Store unbiased data into new csv

In [ ]:
import csv
fp = open('Heart_Health_Age.csv', 'w' , newline='')
myFile = csv.writer(fp)
column_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'Probability']
myFile.writerow(column_names)


for index,row in dataset_orig.iterrows():
    myFile.writerow(row)

# Train new model on unbiased data

In [ ]:
print(dataset_orig.shape)
np.random.seed(0)
## Divide into train,validation,test
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, random_state=0,shuffle = True)

X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

# --- LSR
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
# --- CART
# clf = tree.DecisionTreeClassifier()

# clf.fit(X_train, y_train)
# import matplotlib.pyplot as plt
# y = np.arange(len(dataset_orig_train.columns)-1)
# plt.barh(y,clf.coef_[0])
# plt.yticks(y,dataset_orig_train.columns)
# plt.show()

# print(clf_male.coef_[0])
# y_pred = clf.predict(X_test)
# cnf_matrix_test = confusion_matrix(y_test,y_pred)

# print(cnf_matrix_test)
# TN, FP, FN, TP = confusion_matrix(y_test,y_pred).ravel()


print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'recall'))
print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'far'))
print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'precision'))
print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'accuracy'))
print("aod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'aod'))
print("eod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'eod'))

print("TPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'TPR'))
print("FPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'age', 'FPR'))